In [ ]:
from cgi import test
from random import random
import requests as req
import lxml.etree as le
import typing
from hashlib import md5
from random import random

class Login(object):
    def __init__(self):
        self.headers = {
            'Referer': 'http://eol.cdnu.edu.cn/eol/index.do',
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
            'Host': 'eol.cdnu.edu.cn'
        }
        self.root_url = 'http://eol.cdnu.edu.cn/eol'
        self.post_url = 'http://eol.cdnu.edu.cn/eol/index.do'
        self.login_url = 'http://eol.cdnu.edu.cn/eol/loginCheck.do'
        self.logined_url = 'http://eol.cdnu.edu.cn/eol/personal.do'
        self.logined_flag = False # Failed some call if NOT TRUE
        self.session = req.Session()

    def __md5(self, password:str) -> str:
        func1 = lambda x: md5(x.encode()).hexdigest()
        m1 = func1('v8_blue')
        m2 = func1(password)
        result = func1(m2+m1)
        return result
    def __selector(self, response):
        pass
    def __logintoken(self) -> str:
        response = self.session.get(self.post_url, headers=self.headers)
        selector = le.HTML(response.text)
        logintoken = selector.xpath("//div[@class='loginwrap']//input[1]/@value")[0]
        return logintoken
    def __givemehwlist(self):
        reminder_api_url = self.root_url + \
            "/welcomepage/student/interaction_reminder_v8.jsp?r="+str(random())
        response = self.session.get(reminder_api_url)
        selector = le.HTML(response.text)
        hwlist = selector.xpath("//a[contains(text(),'待提交')]/..//li//a")
        return hwlist

    def login(self, username, password):
        post_data = {
            'logintoken': self.__logintoken(),  # get logintoken from a hidden input in the login form
            'IPT_LOGINUSERNAME': username,  # 
            'IPT_LOGINPASSWORD': self.__md5(password)
        }
        # POST LOGIN requests 
        response = self.session.post(self.login_url, data=post_data, headers=self.headers)
        if response.status_code == 200 and (le.HTML(response.text).xpath("//div[@class='userinfobody']")!=[]):
            print("STATUS CODE: ", response.status_code)
            print("LOGIN SUCCESSED")
            self.logined_flag = True
        else:
            print("STATUS CODE: ", response.status_code)
            print("LOGIN FAILED")

    def hwlist(self):
        # selector = le.HTML(response.text)
        # hwlist = selector.xpath("//div[@class='reminder']//a")
        if self.logined_flag != True:
            print("[Not Login]")
            return 
        return self.__givemehwlist()

    def test_session_get(self):
        response = self.session.get(self.login_url, headers=self.headers)
        return response

    def test_md5(self, password:str) -> str:
        return self.__md5(password)

    def test_ajax(self):
        response = self.session.get("http://eol.cdnu.edu.cn/eol/welcomepage/student/interaction_reminder_v8.jsp?r="+str(random()))
        return response.text